# VLAN 지원 여부 확인

## 테스트 1

### 테스트 환경 
- VMware workstation 11
- CentOS 7 2대 설치
- 각 호스트 NIC는 .vmx 파일에 다음 항목을 추가하여 e1000으로 변경
 - CentOS에서 NIC이 인식 안되어서 변경함
 - ethernet?.virtualDev = "e1000"  (?는 번호 - 1,2,3,...)
- Hostonly로 VMnet12 생성하여 각 호스트에 연결

### 테스트 결과 

- host 1에서 vlan tag 5번 만들고 IP 할당 (192.168.5.11)

```
[root@study-network neutron]# vconfig add eth1 5
Added VLAN with VID == 5 to IF -:eth1:-
[root@study-network neutron]# ifconfig eth1.5 192.168.5.11 netmask 255.255.255.0 broadcast 192.168.5.255 up
```

- host 2에서 vlan tag 5번 만들고 IP 할당 (192.168.5.12)

```
[root@study-compute2 neutron]# vconfig add eth1 5
Added VLAN with VID == 5 to IF -:eth1:-
[root@study-compute2 neutron]# ifconfig eth1.5 192.168.5.12 netmask 255.255.255.0 broadcast 192.168.5.255 up
```

- ping: host 1 -> host 2 ==> X

```
[root@study-network neutron]# ping 192.168.5.12
PING 192.168.5.12 (192.168.5.12) 56(84) bytes of data.
From 192.168.5.11 icmp_seq=1 Destination Host Unreachable
From 192.168.5.11 icmp_seq=2 Destination Host Unreachable
From 192.168.5.11 icmp_seq=3 Destination Host Unreachable
From 192.168.5.11 icmp_seq=4 Destination Host Unreachable
^C
--- 192.168.5.12 ping statistics ---
4 packets transmitted, 0 received, +4 errors, 100% packet loss, time 3000ms
pipe 4
```

- ping: host 2 -> host 1 ==> X

```
[root@study-compute2 neutron]# ping 192.168.5.11
PING 192.168.5.11 (192.168.5.11) 56(84) bytes of data.
From 192.168.5.12 icmp_seq=1 Destination Host Unreachable
From 192.168.5.12 icmp_seq=2 Destination Host Unreachable
From 192.168.5.12 icmp_seq=3 Destination Host Unreachable
From 192.168.5.12 icmp_seq=4 Destination Host Unreachable
^C
--- 192.168.5.11 ping statistics ---
4 packets transmitted, 0 received, +4 errors, 100% packet loss, time 3000ms
pipe 4
```


## 테스트 2

### 테스트 환경
- VirtualBox 4.3.20
- Ubuntu 14.04 2대 설치
- 호스트 전용 네트워크 생성하여 각 호스트에 연결

### 테스트 결과 

- host 1에서 vlan tag 5번 만들고 IP 할당 (192.168.5.11)

```
root@ubjuno-contnet:~# vconfig add eth2 5
Added VLAN with VID == 5 to IF -:eth2:-
root@ubjuno-contnet:~# ifconfig eth2.5 192.168.5.11 netmask 255.255.255.0 broadcast 192.168.5.255 up
```

- host 2에서 vlan tag 5번 만들고 IP 할당 (192.168.5.12)

```
root@ubjuno-compute:~# vconfig add eth2 5
Added VLAN with VID == 5 to IF -:eth2:-
root@ubjuno-compute:~# ifconfig eth2.5 192.168.5.12 netmask 255.255.255.0 broadcast 192.168.5.255 up
```

- ping: host 1 -> host 2 ==> O

```
root@ubjuno-contnet:~# ping -c 4 192.168.5.12
PING 192.168.5.12 (192.168.5.12) 56(84) bytes of data.
64 bytes from 192.168.5.12: icmp_seq=1 ttl=64 time=0.776 ms
64 bytes from 192.168.5.12: icmp_seq=2 ttl=64 time=1.18 ms
64 bytes from 192.168.5.12: icmp_seq=3 ttl=64 time=0.565 ms
64 bytes from 192.168.5.12: icmp_seq=4 ttl=64 time=0.967 ms

--- 192.168.5.12 ping statistics ---
4 packets transmitted, 4 received, 0% packet loss, time 3002ms
rtt min/avg/max/mdev = 0.565/0.874/1.188/0.230 ms
```

- ping: host 2 -> host 1 ==> O

```
root@ubjuno-compute:~# ping -c 4 192.168.5.11
PING 192.168.5.11 (192.168.5.11) 56(84) bytes of data.
64 bytes from 192.168.5.11: icmp_seq=1 ttl=64 time=0.283 ms
64 bytes from 192.168.5.11: icmp_seq=2 ttl=64 time=0.387 ms
64 bytes from 192.168.5.11: icmp_seq=3 ttl=64 time=0.470 ms
64 bytes from 192.168.5.11: icmp_seq=4 ttl=64 time=0.385 ms

--- 192.168.5.11 ping statistics ---
4 packets transmitted, 4 received, 0% packet loss, time 2997ms
rtt min/avg/max/mdev = 0.283/0.381/0.470/0.067 ms
```


## 기타 

- 8021q 모듈이 설치되어 있어야 한다.
 - modprobe 8021q

- tcpdump로 vlan 패킷 확인하는 법 (VLAN acceleration으로 인해 조금 복잡하게 실행해야 함)
 - tcpdump -i [네트워크 인터페이스 카드] -Uw - | tcpdump -en -r - vlan [vlan 태그 번호]
